### Install Libraries

In [1]:
import pandas as pd
import xlsxwriter

### read data and filter

In [2]:
url = "https://github.com/hnawaz007/pythondataanalysis/blob/main/AutomateExcelReports/AdvWorksData.xlsx?raw=true"

In [3]:
# read from url
df=pd.read_excel(url, sheet_name='Sheet1')

In [4]:
category_sales = df.groupby(['productsubcategory','OrderDate'])[['StandardCost','UnitPriceDiscount','UnitPrice','ListPrice','OrderQuantity','Sales']].sum().sort_values(by='Sales', ascending = False).reset_index()
category_sales.head()

,productsubcategory,OrderDate,StandardCost,UnitPriceDiscount,UnitPrice,ListPrice,OrderQuantity,Sales
0,Road Bikes,2012-06-30,548293.4739,11.23,490258.8203,831891.3600,2310,1.599893e+06
1,Road Bikes,2013-03-30,525119.3943,0.15,476680.3665,794663.0250,2084,1.518931e+06
2,Road Bikes,2012-09-30,530267.7827,0.10,481217.0095,802158.8475,2081,1.515724e+06
3,Road Bikes,2012-12-31,495090.1958,0.06,449573.5941,749367.6225,1863,1.345480e+06
4,Mountain Bikes,2011-10-01,321987.1136,0.06,343311.4866,572523.3100,628,1.272613e+06


In [5]:
dsf= category_sales[(category_sales['OrderDate'] == '2014-05-01')]
dsf.head()

,productsubcategory,OrderDate,StandardCost,UnitPriceDiscount,UnitPrice,ListPrice,OrderQuantity,Sales
7,Touring Bikes,2014-05-01,487204.4461,0.04,470212.0236,783790.92,1336,1.192863e+06
43,Mountain Bikes,2014-05-01,233354.6931,31.60,238801.9080,427759.32,897,7.812294e+05
48,Road Bikes,2014-05-01,261755.2104,0.23,246473.7203,411306.13,902,7.420360e+05
95,Mountain Frames,2014-05-01,84150.1992,0.00,92202.5520,153670.92,613,2.209291e+05
103,Touring Frames,2014-05-01,76979.4336,0.00,77056.4880,128427.48,373,1.816296e+05


### Define Conditional Formatting function

In [6]:
def highlight_rows(row):
    value = row.loc['Sales']
    if value < 1000:
        color = '#FFB3BA' # Red
    elif value > 100000:
        color = '#BAFFC9' # Green
    else:
        color = '#FCFCFA' # Grey
    return ['background-color: {}'.format(color) for r in row]

dt = dsf.style.apply(highlight_rows, axis=1)

In [7]:
dt

,productsubcategory,OrderDate,StandardCost,UnitPriceDiscount,UnitPrice,ListPrice,OrderQuantity,Sales
7,Touring Bikes,2014-05-01 00:00:00,487204.446100,0.040000,470212.023600,783790.920000,1336,1192863.491796
43,Mountain Bikes,2014-05-01 00:00:00,233354.693100,31.600000,238801.908000,427759.320000,897,781229.377200
48,Road Bikes,2014-05-01 00:00:00,261755.210400,0.230000,246473.720300,411306.130000,902,742035.995418
95,Mountain Frames,2014-05-01 00:00:00,84150.199200,0.000000,92202.552000,153670.920000,613,220929.060000
103,Touring Frames,2014-05-01 00:00:00,76979.433600,0.000000,77056.488000,128427.480000,373,181629.600000
127,Road Frames,2014-05-01 00:00:00,45025.371100,0.000000,44520.804000,74201.340000,240,102948.012000
171,Jerseys,2014-05-01 00:00:00,7735.481600,0.710000,5990.315100,10046.080000,1088,33427.198227
191,Shorts,2014-05-01 00:00:00,2539.101100,0.420000,4044.022200,6789.030000,577,23403.389181
202,Cranksets,2014-05-01 00:00:00,7036.714000,0.000000,9509.070000,15848.450000,123,20937.762000
207,Vests,2014-05-01 00:00:00,1852.422000,0.560000,2936.240000,4953.000000,557,20275.207100


### Save to excel

In [8]:
dt.to_excel("G:\\test\\test101.xlsx", index=False)

### Method two: XlsxWriter

In [9]:
writer = pd.ExcelWriter(r'G:\\test\\enhancedoutput.xlsx', engine='xlsxwriter')
dsf.to_excel(writer, index=False, sheet_name='report')
#
workbook = writer.book
worksheet = writer.sheets['report']
#Now we have the worksheet object. We can manipulate it 
worksheet.set_zoom(90)

### Set header formating

In [10]:
header_format = workbook.add_format({
        "valign": "vcenter",
        "align": "center",
        "bg_color": "#951F06",
         "bold": True,
        'font_color': '#FFFFFF',
         'border' : 1, 
         'border_color': ''#D3D3D3'
    })

In [11]:
# Full border formatting for conditional formatted cells
full_border = workbook.add_format({ "border" : 1, "border_color": "#D3D3D3"})

### Add Title and Sub-title

In [12]:
#add title
title = "Monthly Sales Report "
#merge cells
format = workbook.add_format()
format.set_font_size(20)
format.set_font_color("#333333")
#
subheader = "Sales report for May"
worksheet.merge_range('A1:AS1', title, format)
worksheet.merge_range('A2:AS2', subheader)
worksheet.set_row(2, 15) # Set the header row height to 15
# puting it all together
# Write the column headers with the defined format.
for col_num, value in enumerate(dsf.columns.values):
    #print(col_num, value)
    worksheet.write(2, col_num, value, header_format)

### Number Formatting

In [13]:
# Add a number format for cells with money.
currency_format = workbook.add_format({'num_format': '$#,##0.00'})
percent_fmt = workbook.add_format({'num_format': '0.0%'})

In [14]:
# Adjust the column width.
worksheet.set_column('A:H', 20)

0

In [15]:
# Numbers formatting
worksheet.set_column('C:C', 12, currency_format)
worksheet.set_column('E:E', 12, currency_format)
worksheet.set_column('F:F', 12, currency_format)
worksheet.set_column('H:H', 12, currency_format)
# Percent formatting
worksheet.set_column('D:D', 20, percent_fmt)


0

### Conditional Formatting

In [16]:
# Light red fill with dark red text.
lessthanthousand = workbook.add_format({'bg_color':   '#FFC7CE','font_color': '#9C0006'})
worksheet.conditional_format('A4:H27', {'type':     'formula','criteria': '=$H4<1000','format':   lessthanthousand})
# add borders
worksheet.conditional_format('A4:H27', {'type':  'formula','criteria': '=$H4<1000','format':   full_border})

0

In [17]:
# Green fill with dark green text.
morethanhundredthousand = workbook.add_format({'bg_color':   '#C6EFCE','font_color': '#006100'})
worksheet.conditional_format('A4:H27', {'type': 'formula','criteria': '=$H4>100000','format':   morethanhundredthousand})
# add  borders
worksheet.conditional_format('A4:H27', {'type':  'formula','criteria': '=$H4>100000','format':   full_border})

0

In [18]:
# Enhanced output
writer.save()